In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hate-speech-and-offensive-language-dataset/labeled_data.csv


In [2]:
import pandas as pd
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
import torch
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
from sklearn.metrics import classification_report

In [3]:
PATH = '../input/hate-speech-and-offensive-language-dataset/'
df = pd.read_csv(PATH+'labeled_data.csv')

In [4]:
import re
def clean_text(text):
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"&[a-z]+;", "", text)
    text = re.sub(r"[^a-zA-Z\s]", "", text)  # Remove special characters
    return text.lower()

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
df['cleanedTweet'] = df['tweet'].apply(clean_text)

In [7]:
tokens = tokenizer(list(df['cleanedTweet']),padding=True,return_tensors="pt",max_length =128)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels = 3)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2888: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
from sklearn.model_selection import train_test_split

train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['cleanedTweet'], df['class'], test_size=0.2, random_state=42
)


In [9]:
train_tokens = tokenizer(list(train_texts), padding=True, return_tensors="pt", max_length=128)
test_tokens = tokenizer(list(test_texts), padding=True, return_tensors="pt", max_length=128)

In [10]:
train_labels = torch.tensor(train_labels.values)
test_labels = torch.tensor(test_labels.values)

In [11]:
class HateSpeechDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}, self.labels[idx]

train_dataset = HateSpeechDataset(train_tokens, train_labels)
test_dataset = HateSpeechDataset(test_tokens, test_labels)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [12]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [13]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch
from sklearn.metrics import accuracy_score

criterion = torch.nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.5, verbose=True)
for epoch in range(10):  # Increased to 10 epochs
    total_loss = 0
    for batch in train_dataloader:
        optimizer.zero_grad()
        inputs, labels = batch
        inputs = {key: val.to(device) for key, val in inputs.items()}
        labels = labels.to(device)

        outputs = model(**inputs, labels=labels)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {total_loss:.4f}")

    # Call the scheduler after each epoch (adjust learning rate based on loss)
    scheduler.step(total_loss)

Epoch 1, Loss: 362.6820
Epoch 2, Loss: 235.4847
Epoch 3, Loss: 132.0161
Epoch 4, Loss: 48.8179
Epoch 5, Loss: 22.9497
Epoch 6, Loss: 14.8121
Epoch 7, Loss: 11.8187
Epoch 8, Loss: 10.6412
Epoch 9, Loss: 13.2112
Epoch 10, Loss: 7.6332


In [14]:
from sklearn.metrics import accuracy_score, classification_report
import re
import os
model.eval()
predictions, true_labels = [], []
with torch.no_grad():
    for batch in test_dataloader:
        inputs, labels = batch
        inputs = {key: val.to(device) for key, val in inputs.items()}
        labels = labels.to(device)

        outputs = model(**inputs)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)

        predictions.extend(preds.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(true_labels, predictions)
print(f"Accuracy: {accuracy:.4f}")


Accuracy: 0.8648


In [15]:
accuracy = accuracy_score(true_labels, predictions)
print(f"Accuracy: {accuracy:.4f}")
print(classification_report(true_labels, predictions))

# Save the model
output_dir = './bert-fine-tuned'
os.makedirs(output_dir, exist_ok=True)
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"Model saved to {output_dir}")

Accuracy: 0.8648
              precision    recall  f1-score   support

           0       0.29      0.50      0.37       290
           1       0.93      0.91      0.92      3832
           2       0.92      0.80      0.85       835

    accuracy                           0.86      4957
   macro avg       0.71      0.73      0.71      4957
weighted avg       0.89      0.86      0.88      4957

Model saved to ./bert-fine-tuned


In [16]:
torch.save(model.state_dict(), "/kaggle/working/bert_model_state_dict.pth")


In [17]:
torch.save(model, "/kaggle/working/bert_model_full.pth")